In [1]:
!pip install Flask Flask-SQLAlchemy



In [2]:
# Flask app setup in Jupyter Notebook

from flask import Flask, render_template_string, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
import threading

# Initialize Flask app and SQLAlchemy
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Define a model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)

    def __repr__(self):
        return f'<User {self.name}>'

# Create database and tables
db.create_all()

# Define routes
@app.route('/')
def index():
    users = User.query.all()
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>User List</title>
    </head>
    <body>
        <h1>User List</h1>
        <a href="{{ url_for('add_user') }}">Add User</a>
        <ul>
            {% for user in users %}
                <li>{{ user.name }} ({{ user.email }})</li>
            {% endfor %}
        </ul>
    </body>
    </html>
    ''', users=users)

@app.route('/add_user', methods=['GET', 'POST'])
def add_user():
    if request.method == 'POST':
        name = request.form.get('name')
        email = request.form.get('email')
        user = User(name=name, email=email)
        db.session.add(user)
        db.session.commit()
        return redirect(url_for('index'))
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Add User</title>
    </head>
    <body>
        <h1>Add User</h1>
        <form action="{{ url_for('add_user') }}" method="POST">
            <label for="name">Name:</label>
            <input type="text" id="name" name="name" required>
            <label for="email">Email:</label>
            <input type="email" id="email" name="email" required>
            <button type="submit">Add</button>
        </form>
        <a href="{{ url_for('index') }}">Back to User List</a>
    </body>
    </html>
    ''')

# Run Flask app in a separate thread
def run_app():
    app.run(port=5000, debug=True, use_reloader=False)

threading.Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Aug/2024 20:06:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2024 20:06:29] "GET /add_user HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2024 20:06:40] "POST /add_user HTTP/1.1" 302 -
127.0.0.1 - - [11/Aug/2024 20:06:40] "GET / HTTP/1.1" 200 -
